## Installs and Imports

In [1]:
%%capture
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/content/drive', force_remount=True)

!pip install openai
import openai
from openai import OpenAI

!pip install pydub

client = OpenAI(
    api_key='your-key-here'
)

In [ ]:
import os

mp3_files = []
m4a_files = []
best_of_files = []

path = '/content/drive/My Drive/'

for x in os.listdir(path):
    if "mp3" in x:
        mp3_files.append(x)
    if "m4a" in x:
        m4a_files.append(x)
    if "best-of" in x:
        best_of_files.append(x)

for x in mp3_files:
    print(x)
print()

for x in m4a_files:
    print(x)
print()

for x in best_of_files:
    print(x)
print()

In [37]:
import os
from pydub import AudioSegment

def combine_audio_segments(input_files, output_file):
    # Create an empty AudioSegment to hold the combined audio
    combined_audio = AudioSegment.empty()

    # Iterate over each input file name
    for file_name in input_files:
        # Construct the full file path

        # Load the input audio file
        audio = AudioSegment.from_file(file_name)
        # Append the audio from this file to the combined audio
        combined_audio += audio

    # Export the combined audio to a new file
    combined_audio.export(output_file, format="wav")

In [39]:
input_files = best_of_files
output_file = path+"best-of-audio"
combine_audio_segments(input_files, output_file)

In [25]:
import os
from pydub import AudioSegment

def split_audio_by_chunks(file_name, input_file, output_dir, num_chunks):
    # Load the audio file
    audio = AudioSegment.from_file(input_file, format="m4a")

    # Calculate the size of each chunk
    chunk_size = len(audio) // num_chunks

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Split the audio into chunks
    for i in range(num_chunks):
        start_byte = i * chunk_size
        end_byte = min((i + 1) * chunk_size, len(audio))
        chunk = audio[start_byte:end_byte]
        output_file = output_dir+f"{file_name}-chunk_{i}.wav"
        print(f"output_file: {output_file}")
        chunk.export(output_file, format="wav")

    print(f"Audio file split into {num_chunks} chunks.")

In [ ]:
# audio_file = client.files.retrieve("file-PxvfyVEfVwTaKTUneWO793GP")

for x in m4a_files:
    file_name = x
    print(f"file_name: {file_name}")
    input_file = path+file_name   # Provide your input audio file
    print(f"input_file: {input_file}")
    chunks_dir = path+file_name+"-chunks/"      # Provide the directory where chunks will be saved
    print(f"chunks_dir: {chunks_dir}")
    max_size_bytes = 26214400 # Maximum size of each chunk in bytes (e.g., 10 MB)
    file_size = os.path.getsize(path+file_name)
    print(f"file_size: {file_size}")
    num_chunks = (int(file_size/max_size_bytes)+2)*4
    print(f"num_chunks: {num_chunks}")
    print()
    split_audio_by_chunks(file_name, input_file, chunks_dir, num_chunks)

In [27]:
# Define function to check if content is inappropriate
def is_content_inappropriate(input_text):
    # Call OpenAI's content moderation model
    moderation = client.moderations.create(input=input_text)

    # Extracting the CategoryScores object from the moderation response
    category_scores = moderation.results[0].category_scores

    # Extracting all categories from the CategoryScores object
    categories = category_scores.__dict__

    # Extracting the category with the highest score
    highest_flag = max(categories, key=lambda x: categories[x])

    # Check if the content is flagged as inappropriate
    if moderation.results[0].flagged:
        # Get the highest flag
        return True, highest_flag
    else:
        return False, None

In [28]:
from pydub import AudioSegment

def stitch_audio_segments(timestamps, input_file, output_file):
    # Load the input audio file
    audio = AudioSegment.from_file(input_file)

    # Check if the output file already exists
    if os.path.exists(output_file):
        # Load the existing output audio file
        stitched_audio = AudioSegment.from_file(output_file)
    else:
        # If the output file doesn't exist, create an empty audio segment
        stitched_audio = AudioSegment.empty()

    # Iterate over each (start, end) timestamp pair
    for start, end in timestamps:
        # Extract the audio segment based on timestamps
        segment = audio[start*1000:end*1000]  # Convert timestamps to milliseconds

        # Append the segment to the stitched audio
        stitched_audio += segment

    # Export the stitched audio to a new file
    stitched_audio.export(output_file, format="wav")

In [ ]:
import os
import re

for x in m4a_files[3:]:
    file_name = x
    print(f"file_name: {file_name}")
    input_file = path+file_name   # Provide your input audio file
    print(f"input_file: {input_file}")
    chunks_dir = path+file_name+"-chunks/"      # Provide the directory where chunks will be saved
    print(f"chunks_dir: {chunks_dir}")
    max_size_bytes = 26214400 # Maximum size of each chunk in bytes (e.g., 10 MB)
    file_size = os.path.getsize(path+file_name)
    print(f"file_size: {file_size}")
    num_chunks = (int(file_size/max_size_bytes)+2)*4
    print(f"num_chunks: {num_chunks}")
    print()

    output_file = path+'best-of-'+file_name

    for chunk_file_name in os.listdir(chunks_dir):

        # print(f"{i} of {len(m4a_files)} m4a_files")

        audio_file = open(chunks_dir+chunk_file_name, "rb")

        file_size = os.path.getsize(chunks_dir+chunk_file_name)

        if file_size > 26214400:
            print(f"Too Big, file_name: {chunk_file_name}, file_size: {file_size}")
            # continue

        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            response_format="verbose_json",
            timestamp_granularities=["segment"]
        )

        # print(f"Length of transcript: {len(transcript.text)}")

        duration = transcript.duration
        segments = transcript.segments

        flagged_segments = []

        for segment in segments:
            id = segment['id']
            start = segment['start']
            end = segment['end']
            section = segment['text']
            #print(f"Start: {start}, End: {end}")

            section = re.sub(r'\b(\w+)\s+\1\b', r'\1', section)
            section = re.sub(r'\b(\w+)\s+\1(?:\s+\1)+\b', r'\1', section)

            judgement, flag = is_content_inappropriate(section)

            if judgement:
                flagged_segments.append(segment)
                print(f"Start: {start}, End: {end}")
                print(f"FLAG: {flag}")
                print(section)
                # print()

        timestamps = []
        for segment in flagged_segments:
            timestamps.append((segment['start'],segment['end']))

        input_file = path+file_name

        stitch_audio_segments(timestamps, chunks_dir+chunk_file_name, output_file)